In [4]:
# csv of verbs - make them into dictionary and put it as the sports verb array - animesh
# function for document folder input 
# proccessed person add as a weight
# faiss approach as a weight
# merge all the codes

# Traverse folder
# Append the sentences, respective of their heaidng
# Ranking for before and after seperately
# rank all together

## Improvements
- Adaptive Weighting: Instead of fixed factors, use a small neural network or regression model to optimize weights.
- Synonym Expansion: Use a thesaurus or word embeddings to ensure better handling of unseen words.
- Dynamic Personality & Keywords List: Regularly update using trending sports events and news.
- Larger TF-IDF Corpus: Instead of computing against just ranking candidates, consider a broader sports news dataset.

# Imports

In [ ]:
pip install nltk numpy faiss-cpu spacy sentence-transformers transformers torch

In [6]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import numpy as np
import pandas as pd
import faiss
import spacy
from sentence_transformers import SentenceTransformer
import spacy
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from tabulate import tabulate
import os
import json

c:\Users\draka\Downloads\Sports_Article_Ranking\virtual_environment\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# For generating Semantic scores

# Using Sentence Semantic Score

In [18]:
# Example sentences: 5 sports-related, 5 non-sports-related
sentences = [
    "Cristiano Ronaldo scored a goal in the final minute.",
    "Serena Williams is practicing her serve on the court.",
    "The basketball team celebrated their championship victory.",
    "The marathon runner finished the race in record time.",
    "The coach is discussing tactics with the players before the match.",
    "Elon Musk announced a new AI breakthrough at Tesla.",
    "The scientist is conducting an experiment in the lab.",
    "A chef is preparing a gourmet meal in the restaurant.",
    "The author is writing a new science fiction novel.",
    "Tourists are exploring ancient ruins in Greece."
]

# Process each sentence and store the results
#scores = {sentence: process_sentence(sentence, verb_dict, model, index, verbs, nlp, k=1) for sentence in sentences}

scores = [process_sentence(sentence, verb_dict, embedding_model, index, verbs, nlp, k=1) for sentence in sentences]

# Print the scores for comparison
for i in range(len(scores)):
    print(f"Sentence: {sentences[i]}\nScore: {scores[i]}\n")


Sentence: Cristiano Ronaldo scored a goal in the final minute.
Score: 0.82

Sentence: Serena Williams is practicing her serve on the court.
Score: 0.7175

Sentence: The basketball team celebrated their championship victory.
Score: 0.86375

Sentence: The marathon runner finished the race in record time.
Score: 0.7889999999999999

Sentence: The coach is discussing tactics with the players before the match.
Score: 0.695

Sentence: Elon Musk announced a new AI breakthrough at Tesla.
Score: 0.6900000000000001

Sentence: The scientist is conducting an experiment in the lab.
Score: 0.6359999999999999

Sentence: A chef is preparing a gourmet meal in the restaurant.
Score: 0.5527272727272727

Sentence: The author is writing a new science fiction novel.
Score: 0.426

Sentence: Tourists are exploring ancient ruins in Greece.
Score: 0.48571428571428577



# Sentiment Score

# Sentiment Use case

In [20]:
# Example usage
sentences = [
    "Ireland's quicks made early inroads after their batters got them to a competitive total",
    "Donald Trump ,Narendra Modi , Joe Biden met basketball player Michael Jordan in an exciting match",
    "Jason Smith's 91 was in vain as South Africa lost to Ireland for only the second time in ODIs.Graham Hume quickly settled into his work with the new ball, picking up a wicket.Mark Adair took two wickets in a probing opening spell, Craig Young and Graham Hume claimed three each and spinner Matthew Humphreys’ frugal seven overs brought him 1-16 as the Proteas were dismissed for 215  ",
    "Jason Smith's 91 was in vain as South Africa lost to Ireland for only the second time in ODIs",
    "Stirling and Balbirnie's 101-run stand was broken when Balbirnie holed out as he miscued a pull off Williams",
    "Paul Stirling made 88 from 92 deliveries",
    "Harry Tector's rapid 60 from 48 was a welcome return to form",
    "Kyle Verreynne (38) looked to find a way back for his team but couldn’t build on his start",
    "Jason Smith and Andile Phehlukwayo frustrated Ireland until the latter top-edged a pull shot to Young",
    "Bjorn Fortuin (11) was bowled by Matthew Humphreys and Williams became Young’s third wicket",
    "Smith moved up a gear and was closing in on a century until he gave a chance to deep third off the bowling of Hume",
    "Hume picked up that wicket in the 47th over, delivering the perfect end to Ireland Men’s international year",
    "South Africa’s top order struggled as the new ball again zipped around under the lights, and were quickly 10-3 and effectively beaten",
    "Mark Adair took two wickets in a probing opening spell, Craig Young and Graham Hume claimed three each and spinner Matthew Humphreys’ frugal seven overs brought him 1-16 as the Proteas were dismissed for 215",
    "Paul Stirling top-scored with 88 from 92 balls",
    "Jason Smith struck a defiant 91 from 93 balls",
    "Ryan Rickelton (4) caught at slip by Stirling in the first over",
    "Reeza Hendricks (1) thick-edged Graham Hume’s outswinger to third slip",
    "Rassie van der Dussen trapped plumb lbw by Mark Adair",
    "Kyle Verreynne’s spirited counterattack yielded 38 from 36 balls",
    "Craig Young’s outswinger found Tristan Stubbs’ edge for keeper Lorcan Tucker",
    "Andile Phehlukwayo (23 from 28 balls) briefly threatened a counter-attack",
    "Curtis Campher made a lively 34 from 36 balls",
    "Harry Tector ran hard during his fine innings of 60 from 48 deliveries",
    "Graham Hume to Lungi Ngidi. Half volley, wide outside off stump on the front foot Slog, outside edge, caught by Tucker.",
    "Wide Graham Hume to Lungi Ngidi. Yorker, wide outside off stump on the front foot cutting, missed for 1 run, fielded by Tucker.",
    "APPEAL! Matthew Humphreys to Ottniel Baartman. Quicker length ball, outside off stump on the front foot driving, hit pad to silly point for no runs, fielded by Tucker, appeal made for L.B.W.",
    "FOUR! Matthew Humphreys to Lungi Ngidi. Stock ball half volley, to leg on the front foot Slog, well timed past long on for 4 runs."
    ]

# Get intensity scores
intensity_scores = detect_emotion_intensity(sentences)

# Print the scores for comparison
for i in range(len(intensity_scores)):
    print(f"Sentence: {sentences[i]}\nIntensity Score: {intensity_scores[i]}\n")

Sentence: Ireland's quicks made early inroads after their batters got them to a competitive total
Intensity Score: 0.9320802092552185

Sentence: Donald Trump ,Narendra Modi , Joe Biden met basketball player Michael Jordan in an exciting match
Intensity Score: 0.6569411754608154

Sentence: Jason Smith's 91 was in vain as South Africa lost to Ireland for only the second time in ODIs.Graham Hume quickly settled into his work with the new ball, picking up a wicket.Mark Adair took two wickets in a probing opening spell, Craig Young and Graham Hume claimed three each and spinner Matthew Humphreys’ frugal seven overs brought him 1-16 as the Proteas were dismissed for 215  
Intensity Score: 0.6778813600540161

Sentence: Jason Smith's 91 was in vain as South Africa lost to Ireland for only the second time in ODIs
Intensity Score: 0.669975757598877

Sentence: Stirling and Balbirnie's 101-run stand was broken when Balbirnie holed out as he miscued a pull off Williams
Intensity Score: 0.8278850317

# Sarcasm

In [22]:
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import T5Tokenizer
from transformers import AutoModelForSeq2SeqLM
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
from functools import reduce
import operator
import math

In [26]:
sarcastic_sentences = [
    "Oh great, another meeting that could have been an email.",
    "Wow, because I totally needed more traffic on my way home.",
    "Oh sure, I'd love to work overtime for free!",
    "Fantastic, my phone just died when I needed it the most.",
    "Oh wonderful, the WiFi is down again. Best timing ever!",
    "Yes, because standing in a long queue is my favorite hobby.",
    "Oh, I just love it when my alarm doesn’t go off.",
    "Wow, another software update! My day just got better.",
    "Amazing, I totally enjoy getting unsolicited advice.",
    "Oh great, another robocall! How did I get so lucky?"
]

for sentence in sarcastic_sentences:
    print(sentence, "->", is_sarcasm(sentence))

c:\Users\draka\Downloads\Sports_Article_Ranking\virtual_environment\lib\site-packages\transformers\models\t5\tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Oh great, another meeting that could have been an email. -> TRUE
Wow, because I totally needed more traffic on my way home. -> FALSE
Oh sure, I'd love to work overtime for free! -> TRUE
Fantastic, my phone just died when I needed it the most. -> FALSE
Oh wonderful, the WiFi is down again. Best timing ever! -> TRUE
Yes, because standing in a long queue is my favorite hobby. -> TRUE
Oh, I just love it when my alarm doesn’t go off. -> FALSE
Wow, another software update! My day just got better. -> FALSE
Amazing, I totally enjoy getting unsolicited advice. -> TRUE
Oh great, another robocall! How did I get so lucky? -> FALSE


# TF-IDF

# Use TFIDF func

In [29]:
scores = tfidf_sums(sentences)
for i in range(len(scores)):
    print(f"Sentence: {sentences[i]}\nScore: {scores[i]}\n")


Sentence: Ireland's quicks made early inroads after their batters got them to a competitive total
Score: 0.19364440146137588

Sentence: Donald Trump ,Narendra Modi , Joe Biden met basketball player Michael Jordan in an exciting match
Score: 0.2572554639409834

Sentence: Jason Smith's 91 was in vain as South Africa lost to Ireland for only the second time in ODIs.Graham Hume quickly settled into his work with the new ball, picking up a wicket.Mark Adair took two wickets in a probing opening spell, Craig Young and Graham Hume claimed three each and spinner Matthew Humphreys’ frugal seven overs brought him 1-16 as the Proteas were dismissed for 215  
Score: 1.0

Sentence: Jason Smith's 91 was in vain as South Africa lost to Ireland for only the second time in ODIs
Score: 0.31717510164749263

Sentence: Stirling and Balbirnie's 101-run stand was broken when Balbirnie holed out as he miscued a pull off Williams
Score: 0.27354939985914134

Sentence: Paul Stirling made 88 from 92 deliveries
Sc

# Buzzword Score

# NER People score

# Dummy Testing Functions

In [ ]:
# score_columns = {
#     0: "Semantic Score",
#     1: "Sentiment Score",
#     2: "TF-IDF Score",
#     3: "Buzzword Score",
#     4: "NER People Score",
#     5: "Sarcasm Score"
# }

In [ ]:
# def process_sentences(sentences, batch_size=100):
#     results = []
#     for i in range(0, len(sentences), batch_size):
#         batch = sentences[i:i+batch_size]
        
#         semantic_scores = [process_sentence(s, verb_dict, embedding_model, index, verbs, nlp) for s in batch]
#         sentiment_scores_list = detect_emotion_intensity(batch)
#         tfidf_scores = tfidf_sums(batch)
#         buzzword_scores = [calculate_buzzword_score(s, sports_keywords) for s in batch]
#         ner_people_scores = [calculate_ner_people_score(s, nlp, famous_people) for s in batch]
#         sarcasm_scores = [get_sarcasm_score([s])[0] for s in batch]

#         for j, sentence in enumerate(batch):
#             results.append({
#                 'Sentence': sentence,
#                 'Semantic Score': semantic_scores[j],
#                 'Sentiment Score': sentiment_scores_list[j],
#                 'TF-IDF Score': tfidf_scores[j],
#                 'Buzzword Score': buzzword_scores[j],
#                 'NER People Score': ner_people_scores[j],
#                 'Sarcasm Score': sarcasm_scores[j]
#             })

#     return pd.DataFrame(results)

# # Example usage
# sentences = [
#     "Ireland's quicks made early inroads after their batters got them to a competitive total",
#     "Donald Trump ,Narendra Modi , Joe Biden met basketball player Michael Jordan in an exciting match",
#     "Paul Stirling made 88 from 92 deliveries",
#     "Harry Tector's rapid 60 from 48 was a welcome return to form",

#     ]

# results_df = process_sentences(sentences)
# # print(results_df)
# # Display results
# print(tabulate(results_df, headers='keys', tablefmt='pretty', showindex=False))

+---------------------------------------------------------------------------------------------------+--------------------+--------------------+--------------------+----------------+------------------+---------------------+
|                                             Sentence                                              |   Semantic Score   |  Sentiment Score   |    TF-IDF Score    | Buzzword Score | NER People Score |    Sarcasm Score    |
+---------------------------------------------------------------------------------------------------+--------------------+--------------------+--------------------+----------------+------------------+---------------------+
|      Ireland's quicks made early inroads after their batters got them to a competitive total      | 0.8064285714285714 | 0.9627437591552734 | 0.7754664536809429 |   0.94229392   |       0.0        | 0.16519285714285714 |
| Donald Trump ,Narendra Modi , Joe Biden met basketball player Michael Jordan in an exciting match | 0.7866

In [ ]:
# # This function takes a list of sentences and returns a matrix of scores
# # Each row corresponds to a sentence, and each column corresponds to a score type

# import numpy as np

# def create_score_matrix(sentences):
#     scores_matrix = []

#     semantic_scores = [process_sentence(s, verb_dict, embedding_model, index, verbs, nlp) for s in sentences]
#     sentiment_scores_list = detect_emotion_intensity(sentences)
#     tfidf_scores = tfidf_sums(sentences)
#     buzzword_scores = [calculate_buzzword_score(s, sports_keywords) for s in sentences]
#     ner_people_scores = [calculate_ner_people_score(s, nlp, famous_people) for s in sentences]
#     sarcasm_scores = [get_sarcasm_score([s])[0] for s in sentences]

#     for i in range(len(sentences)):
#         scores_matrix.append([
#             semantic_scores[i],
#             sentiment_scores_list[i],
#             tfidf_scores[i],
#             buzzword_scores[i],
#             ner_people_scores[i],
#             sarcasm_scores[i]
#         ])

#     return np.array(scores_matrix)

# # Example usage
# print(create_score_matrix(sentences))


[[0.80642857 0.96274376 0.1936444  0.94229392 0.         0.16519286]
 [0.78666667 0.96895885 0.25725546 0.18845878 3.27917711 0.46303125]
 [0.70981481 0.96757376 1.         2.08809258 0.         0.        ]
 [0.62916667 0.96495014 0.3171751  0.38054177 0.         0.58596842]
 [0.83764706 0.95954639 0.2735494  0.17494943 0.         0.45021667]
 [0.86666667 0.96641338 0.         0.47114696 0.         0.        ]
 [0.6175     0.96764529 0.13013828 0.17132617 0.         0.        ]
 [0.84777778 0.95041317 0.26130408 0.75222438 0.         0.        ]
 [0.76461538 0.96969807 0.27034844 0.50475325 0.39557183 0.49378125]
 [0.82       0.96784168 0.22158904 0.2916624  0.         0.        ]
 [0.676      0.96601892 0.40878339 0.98648519 0.         0.        ]
 [0.73933333 0.96737176 0.2977917  0.55490642 0.         0.        ]
 [0.721      0.9661926  0.37830329 0.94229392 0.         0.        ]
 [0.7136     0.96781933 0.63644274 1.51909202 0.         0.        ]
 [0.96666667 0.96374512 0.07252729

# Final implementation for weights using SGD

In [3]:
import numpy as np

def create_score_matrix(sentences):
    """
    Computes a score matrix for a list of sentences.
    
    Each row corresponds to a sentence and columns represent scores:
      [Semantic, Sentiment, TF-IDF, Buzzword, NER People, Sarcasm]
    
    Assumes the helper functions and global variables below are defined elsewhere:
      - process_sentence(s, verb_dict, embedding_model, index, verbs, nlp)
      - detect_emotion_intensity(sentences)
      - tfidf_sums(sentences)
      - calculate_buzzword_score(s, sports_keywords)
      - calculate_ner_people_score(s, nlp, famous_people)
      - get_sarcasm_score([s])
      
    Parameters:
      - sentences: List of sentence strings.
      
    Returns:
      - A NumPy array representing the score matrix.
    """
    scores_matrix = []

    # Ensure all sentences are strings
    sentences = sanitize_sentences(sentences)

    semantic_scores = [process_sentence(s, verb_dict, embedding_model, index, verbs, nlp) for s in sentences]
    sentiment_scores_list = detect_emotion_intensity(sentences)
    tfidf_scores = tfidf_sums(sentences)
    buzzword_scores = [calculate_buzzword_score(s, sports_keywords) for s in sentences]
    ner_people_scores = [calculate_ner_people_score(s, nlp, famous_people) for s in sentences]
    sarcasm_scores = [get_sarcasm_score([s])[0] for s in sentences]

    for i in range(len(sentences)):
        scores_matrix.append([
            semantic_scores[i],
            sentiment_scores_list[i],
            tfidf_scores[i],
            buzzword_scores[i],
            ner_people_scores[i],
            sarcasm_scores[i]
        ])

    return np.array(scores_matrix)


In [51]:
import numpy as np

def compute_score(item, weights):
    """
    Computes the weighted sum of features for a given score vector.
    
    Parameters:
      - item: A NumPy array representing one score vector.
      - weights: A NumPy array representing the weights.
    
    Returns:
      - The dot product (weighted sum) of the score vector and weights.
    """
    return np.dot(item, weights)

def train_model(data, weights=None, epochs=3, learning_rate=0.01, margin=1.0):
    """
    Trains the model on ranked data using pairwise hinge loss.
    
    Parameters:
      - data: List of ranked lists; each element is an array of 6-dimensional score vectors.
      - weights: Initial weights (NumPy array with 6 elements). If None, uses equal weights.
      - epochs: Number of training epochs.
      - learning_rate: Learning rate for weight updates.
      - margin: Margin value for hinge loss.
      
    Returns:
      - Updated weights as a NumPy array.
    """
    if weights is None:
        weights = np.ones(6) / 6.0  # Initialize equal weights

    for epoch in range(epochs):
        for datapoint in data:
            # Process each adjacent pair in the ranked list
            for i in range(len(datapoint) - 1):
                score_i = compute_score(datapoint[i], weights)
                score_j = compute_score(datapoint[i + 1], weights)
                loss = max(0, margin - (score_i - score_j))
                if loss > 0:
                    grad = -datapoint[i] + datapoint[i + 1]
                    weights = weights - learning_rate * grad
                    # Normalize weights so that they sum to 1
                    weights = weights / np.sum(weights)
        print(f"Epoch {epoch + 1}: Weights = {weights}")
    return weights


In [52]:
import os
import json
import numpy as np

def process_gpt40_folder(root_folder, initial_weight=None, epochs_per_match=1, learning_rate=0.01, margin=1.0):
    """
    Traverses the folder structure starting from root_folder.
    
    Expected folder structure:
      root_folder (e.g., "GPT4o")
        ├── MLB
        ├── NBA
        ├── Odi
        ├── Soccer
        └── T20
    
    For each sport folder, it goes into each match folder, then into the "ranked_gpt" folder.
    Inside "ranked_gpt", it processes each subfolder (in sorted order) and reads the JSON file
    (expected to be a list of sentences) from both the "before" and "after" folders.
    
    The collected sentences are then used to create a score matrix via create_score_matrix.
    This matrix is wrapped in a list (to form a ranked list) and passed to train_model,
    which updates the weights.
    
    Parameters:
      - root_folder: Path to the root folder (e.g., "GPT4o").
      - initial_weight: Initial 6-dimensional weights (NumPy array). If None, defaults to equal weights.
      - epochs_per_match: Number of training epochs to run per match.
      - learning_rate: Learning rate for weight updates.
      - margin: Margin value for hinge loss.
      
    Returns:
      - The final updated weights (NumPy array) after processing all match folders.
    """
    if initial_weight is None:
        current_weights = np.ones(6) / 6.0
    else:
        current_weights = initial_weight

    sports = ['MLB', 'NBA', 'Odi', 'Soccer', 'T20i']

    for sport in sports:
        sport_path = os.path.join(root_folder, sport)
        if not os.path.isdir(sport_path):
            print(f"Sport folder not found: {sport_path}")
            continue

        # Iterate over match folders within the sport folder.
        for match in sorted(os.listdir(sport_path)):
            match_path = os.path.join(sport_path, match)
            if not os.path.isdir(match_path):
                continue

            ranked_gpt_path = os.path.join(match_path, 'ranked_gpt')
            if not os.path.isdir(ranked_gpt_path):
                print(f"'ranked_gpt' folder not found in match folder: {match_path}")
                continue

            sentences = []
            # Process each subfolder in 'ranked_gpt' (sorted order)
            for subfolder in sorted(os.listdir(ranked_gpt_path)):
                subfolder_path = os.path.join(ranked_gpt_path, subfolder)
                if not os.path.isdir(subfolder_path):
                    continue

                # Process only the "before" and "after" folders.
                for timing in ['before', 'after']:
                    timing_folder = os.path.join(subfolder_path, timing)
                    if not os.path.isdir(timing_folder):
                        continue

                    # Find JSON files in the timing folder.
                    json_files = sorted([f for f in os.listdir(timing_folder) if f.endswith('.json')])
                    if not json_files:
                        print(f"No JSON file found in: {timing_folder}")
                        continue

                    # Assume one JSON file per timing folder.
                    json_path = os.path.join(timing_folder, json_files[0])
                    try:
                        with open(json_path, 'r') as jf:
                            data = json.load(jf)
                        # Expecting data to be a list of sentences.
                        if isinstance(data, list):
                            sentences.extend(data)
                        else:
                            print(f"Unexpected JSON format in {json_path}. Expected a list.")
                    except Exception as e:
                        print(f"Error reading {json_path}: {e}")

            if sentences:
                # Create the score matrix from the collected sentences.
                score_matrix = create_score_matrix(sentences)
                # Wrap the score matrix as a ranked list for training.
                training_data = [score_matrix]
                # Update the weights using the training function.
                current_weights = train_model(
                    training_data,
                    weights=current_weights,
                    epochs=epochs_per_match,
                    learning_rate=learning_rate,
                    margin=margin
                )
                print(f"Updated weights after processing match '{match}' in sport '{sport}': {current_weights}\n")
            else:
                print(f"No sentences found for match '{match}' in sport '{sport}'.\n")

    return current_weights


In [ ]:
# root_folder = r"C:\Users\draka\Downloads\Sports_Article_Ranking\GPT4o\GPT4o"
# final_weights = process_gpt40_folder(root_folder)
# print("Final Weights:", final_weights)

c:\Users\draka\Downloads\Sports_Article_Ranking\virtual_environment\lib\site-packages\transformers\models\t5\tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Epoch 1: Weights = [0.16325231 0.16398212 0.17081721 0.17260246 0.16372879 0.16561711]
Updated weights after processing match 'Boston Red Sox vs Minnesota Twins mlb 2024-09-21' in sport 'MLB': [0.16325231 0.16398212 0.17081721 0.17260246 0.16372879 0.16561711]

Epoch 1: Weights = [0.16454357 0.16462951 0.17522641 0.16788127 0.16427864 0.1634406 ]
Updated weights after processing match 'Chicago Cubs vs Cincinnati Reds mlb 2024-09-28' in sport 'MLB': [0.16454357 0.16462951 0.17522641 0.16788127 0.16427864 0.1634406 ]

Epoch 1: Weights = [0.1616741  0.16193038 0.17988154 0.17049129 0.16152744 0.16449524]
Updated weights after processing match 'Chicago White Sox vs Texas Rangers mlb 2024-08-28' in sport 'MLB': [0.1616741  0.16193038 0.17988154 0.17049129 0.16152744 0.16449524]

Epoch 1: Weights = [0.16106061 0.16071081 0.18115752 0.17058925 0.16032011 0.1661617 ]
Updated weights after processing match 'Cincinnati Reds vs Pittsburgh Pirates mlb 2024-09-22' in sport 'MLB': [0.16106061 0.1607

# Testing of weights

In [32]:
def test(sentences):
    """
    Creates a score matrix from sentences, computes final scores using final weights,
    ranks the sentences in descending order based on their scores, and prints the ranking.
    
    Returns:
      final_scores: NumPy array of computed scores.
      ranked_indices: Indices of sentences sorted by score (descending).
      ranked_sentences: List of sentences sorted by rank.
    """
    # Get score matrix: each row corresponds to one sentence with 6 feature scores.
    score_matrix = create_score_matrix(sentences)
    
    # Final weights (normalized so that sum(weights)=1)
    final_weights = np.array([0.21050163, 0.16315615, 0.17638323, 0.10372753, 0.18131574, 0.16491572])
    
    # Compute final score for each sentence (weighted sum of feature scores)
    final_scores = score_matrix.dot(final_weights)
    
    # Rank sentence indices in descending order (higher score is better)
    ranked_indices = final_scores.argsort()[::-1]
    
    # Order sentences based on ranking
    ranked_sentences = [sentences[idx] for idx in ranked_indices]
    
    # Print ranking information: rank, sentence, and score
    for rank, idx in enumerate(ranked_indices, start=1):
        print(f"Rank {rank}: Sentence: '{sentences[idx]}' - Score: {final_scores[idx]:.4f}")
    
    return final_scores, ranked_indices, ranked_sentences

# Example usage:
# sentences = ["Sentence one text.", "Sentence two text.", "Sentence three text.", ...]
# final_scores, ranking_order, ranked_sentences = test(sentences)


In [71]:
test1 = [
    "A win would have been enough for South Africa to become the first team from Group 2 and third overall to progress to the semifinals of the ICC Men’s T20 World Cup 2022.",
    "The Temba Bavuma-led South Africa were the clear favourites but in the unlikely scenario of their defeat, they would have to pack their bags for home.",
    "Netherlands were currently languishing at the bottom of the pile and were already out of the race.",
    "Perhaps the only confidence one could garner this time was it was minnows Netherlands they had to beat which looked rather possible to progress to the semi-finals of the T20 World Cup."
]

final_scores, ranking_order, ranked_sentences = test(test1)

c:\Users\draka\Downloads\Sports_Article_Ranking\virtual_environment\lib\site-packages\transformers\models\t5\tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Rank 1: Sentence: 'A win would have been enough for South Africa to become the first team from Group 2 and third overall to progress to the semifinals of the ICC Men’s T20 World Cup 2022.' - Score: 0.7061
Rank 2: Sentence: 'Perhaps the only confidence one could garner this time was it was minnows Netherlands they had to beat which looked rather possible to progress to the semi-finals of the T20 World Cup.' - Score: 0.5055
Rank 3: Sentence: 'The Temba Bavuma-led South Africa were the clear favourites but in the unlikely scenario of their defeat, they would have to pack their bags for home.' - Score: 0.4390
Rank 4: Sentence: 'Netherlands were currently languishing at the bottom of the pile and were already out of the race.' - Score: 0.2553


In [69]:
test2 = [
    "Sri Lanka won by 5 wickets.",
    "Sri Lanka beat West Indies by 5 wickets",
    "Maheesh Theekshana was named Player of the Match for his bowling figures of 3/25.",
    "Charith Asalanka was named Player of the Series.",
    "Wanindu Hasaranga took 4 wickets for 40 runs.",
    "Maheesh Theekshana took 3 wickets for 25 runs.",
    "Charith Asalanka scored 62 not out off 61 balls for Sri Lanka.",
    "Sherfane Rutherford scored 80 runs off 82 balls for West Indies.",
    "Asitha Fernando took 3 wickets for 35 runs.",
    "Gudakesh Motie scored 50 runs not out off 61 balls for West Indies.",
    "Charith Asalanka scored 62 runs off 61 balls",
    "Nishan Madushka scored 38 runs off 44 balls",
    "Alzarri Joseph took 2 wickets for 30 runs",
    "Gudakesh Motie took 1 wicket for 18 runs"
]

final_scores, ranking_order, ranked_sentences = test(test2)

c:\Users\draka\Downloads\Sports_Article_Ranking\virtual_environment\lib\site-packages\transformers\models\t5\tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Rank 1: Sentence: 'Gudakesh Motie scored 50 runs not out off 61 balls for West Indies.' - Score: 0.5534
Rank 2: Sentence: 'Maheesh Theekshana was named Player of the Match for his bowling figures of 3/25.' - Score: 0.5402
Rank 3: Sentence: 'Charith Asalanka scored 62 not out off 61 balls for Sri Lanka.' - Score: 0.4924
Rank 4: Sentence: 'Sri Lanka beat West Indies by 5 wickets' - Score: 0.4578
Rank 5: Sentence: 'Charith Asalanka was named Player of the Series.' - Score: 0.4522
Rank 6: Sentence: 'Maheesh Theekshana took 3 wickets for 25 runs.' - Score: 0.4467
Rank 7: Sentence: 'Sherfane Rutherford scored 80 runs off 82 balls for West Indies.' - Score: 0.4446
Rank 8: Sentence: 'Sri Lanka won by 5 wickets.' - Score: 0.4419
Rank 9: Sentence: 'Asitha Fernando took 3 wickets for 35 runs.' - Score: 0.4393
Rank 10: Sentence: 'Wanindu Hasaranga took 4 wickets for 40 runs.' - Score: 0.4391
Rank 11: Sentence: 'Gudakesh Motie took 1 wicket for 18 runs' - Score: 0.4390
Rank 12: Sentence: 'Alzarri J

In [70]:
test3 = [
    "The Boston Red Sox will take on the Minnesota Twins in MLB action at Fenway Park on Saturday, beginning at 9:10am AEST.",
    "Richard Fitts (0-0, 0.00 ERA) will start for the Red Sox, while the Twins will counter with David Festa (2-6, 5.07 ERA).",
    "Stats Insider's predictive analytics model gives the Red Sox a 51% chance of beating the Twins at Fenway Park.",
    "The Red Sox are listed as 1.5-run underdogs against the Twins, with odds of $1.60 available at TAB.",
    "For the favourite Twins (-1.5) to cover the line, TAB offers odds of $2.35.",
    "According to Stats Insider's analytics model, the Red Sox (+1.5) are predicted to cover the line 63% of the time, while the 8.5-run over/under is expected to go over 54% of the time.",
    "The over/under for total runs scored is set at 8.5 with TAB, which has odds of $1.91 for the over and $1.91 for the under."
]

final_scores, ranking_order, ranked_sentences = test(test3)

c:\Users\draka\Downloads\Sports_Article_Ranking\virtual_environment\lib\site-packages\transformers\models\t5\tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Rank 1: Sentence: 'The Boston Red Sox will take on the Minnesota Twins in MLB action at Fenway Park on Saturday, beginning at 9:10am AEST.' - Score: 0.5557
Rank 2: Sentence: 'According to Stats Insider's analytics model, the Red Sox (+1.5) are predicted to cover the line 63% of the time, while the 8.5-run over/under is expected to go over 54% of the time.' - Score: 0.5196
Rank 3: Sentence: 'Richard Fitts (0-0, 0.00 ERA) will start for the Red Sox, while the Twins will counter with David Festa (2-6, 5.07 ERA).' - Score: 0.5050
Rank 4: Sentence: 'Stats Insider's predictive analytics model gives the Red Sox a 51% chance of beating the Twins at Fenway Park.' - Score: 0.4649
Rank 5: Sentence: 'The over/under for total runs scored is set at 8.5 with TAB, which has odds of $1.91 for the over and $1.91 for the under.' - Score: 0.4352
Rank 6: Sentence: 'The Red Sox are listed as 1.5-run underdogs against the Twins, with odds of $1.60 available at TAB.' - Score: 0.4016
Rank 7: Sentence: 'For the

# NDCG@5,10 and Recall@5,10

In [ ]:
import numpy as np

def compute_relevance_dict(ground_truth):
    """
    Compute a relevance dictionary from the ground truth ranking.
    The top sentence gets the highest relevance.
    For example, if there are N sentences, the sentence at position i gets relevance: N - i.
    """
    N = len(ground_truth)
    relevance = {sentence: (N - i) for i, sentence in enumerate(ground_truth)}
    return relevance

def dcg(scores, k):
    """
    Compute the Discounted Cumulative Gain (DCG) up to rank k.
    scores: list of relevance scores in the order of the system ranking.
    """
    return sum((2 ** score - 1) / np.log2(i + 2) for i, score in enumerate(scores[:k]))

def compute_ndcg_at_k(system_rank, ground_truth, k):
    """
    Compute NDCG@k given the system ranking and ground truth ranking.
    
    Parameters:
      system_rank: List of sentences as ranked by the system.
      ground_truth: List of sentences as ranked in the ground truth (best first).
      k: Rank cutoff (e.g., 5 or 10).
    
    Returns:
      NDCG score at cutoff k.
    """
    relevance_dict = compute_relevance_dict(ground_truth)
    # Get the relevance scores for the system ranking order.
    system_rels = [relevance_dict.get(sentence, 0) for sentence in system_rank]
    
    dcg_k = dcg(system_rels, k)
    
    # For the ideal ranking, sort the relevance scores in descending order.
    ideal_rels = sorted(relevance_dict.values(), reverse=True)
    idcg_k = dcg(ideal_rels, k)
    
    return dcg_k / idcg_k if idcg_k > 0 else 0.0

def compute_recall_at_k(system_rank, ground_truth, k):
    """
    Compute Recall@k.
    
    Here, we treat the top k sentences in the ground truth as the relevant set.
    
    Parameters:
      system_rank: List of sentences as ranked by the system.
      ground_truth: List of sentences as ranked in the ground truth (best first).
      k: Rank cutoff (e.g., 5 or 10).
    
    Returns:
      Recall score at cutoff k.
    """
    # The relevant set is the top-k sentences in the ground truth.
    relevant_set = set(ground_truth[:k])
    # The retrieved set is the system's top-k.
    retrieved_set = set(system_rank[:k])
    
    recall = len(retrieved_set.intersection(relevant_set)) / float(k)
    return recall

def evaluate_ranking(system_rank, ground_truth):
    """
    Evaluate the system ranking against the ground truth by computing:
      - NDCG@5, NDCG@10,
      - Recall@5, Recall@10.
    
    Parameters:
      system_rank: List of sentences ranked by your system.
      ground_truth: List of sentences from your ground truth (already ranked).
    
    Returns:
      Dictionary with keys 'NDCG@5', 'NDCG@10', 'Recall@5', 'Recall@10'.
    """
    metrics = {}  
    metrics['NDCG@5'] = compute_ndcg_at_k(system_rank, ground_truth, 5)
    metrics['NDCG@10'] = compute_ndcg_at_k(system_rank, ground_truth, 10)
    metrics['Recall@5'] = compute_recall_at_k(system_rank, ground_truth, 5)
    metrics['Recall@10'] = compute_recall_at_k(system_rank, ground_truth, 10)
    return metrics

# --- Example usage ---
# Assume testing_sentence is your ground truth ranking (best first)
# and you obtain system ranking from your test() function.
#
# Example:
# sentences = ["Sentence one text.", "Sentence two text.", "Sentence three text.", ...]
# final_scores, ranking_order, ranked_sentences = test(sentences)
#
# Now evaluate:
# metrics = evaluate_ranking(ranked_sentences, testing_sentence)
# print("Evaluation Metrics:")
# for key, value in metrics.items():
#     print(f"{key}: {value:.4f}")


# 10 matches testing

In [ ]:
# India vs England Odi
testing_sentence_1 = [
    "India and England will face each other in a blockbuster match on October 29, 2023, at the Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium in Lucknow.",
    "India were unbeaten with five wins in as many matches before the game.",
    "The match is expected to be a tough contest as England is the current World Champion, while India has won the World Cup twice.",
    "Rohit Sharma was setting the tone in the powerplay, striking at 134.01.",
    "England has lost four out of their five matches so far and is almost out of the race for the semi-finals, whereas India has won all five of their matches.",
    "Experts predicted India as the favorites given their home advantage.",
    "The Kohli vs. Buttler battle was highlighted as a key contest.",
    "Virat Kohli aims to score a century to equal Sachin Tendulkar's record.",
    "Jasprit Bumrah had an economy of 2.90 in the first ten overs.",
    "Mohammed Siraj was among the most lethal ODI bowlers coming into the World Cup with 30 wickets in 14 matches in 2023.",
    "Historically, England has a better record against India in World Cup matches, winning four out of eight encounters, with India winning three and one match ending in a tie.",
    "Kuldeep Yadav will be crucial in silencing England's aggressive batsmen.",
    "Captain Rohit Sharma emphasized India's focus on maintaining their winning streak.",
    "Jos Buttler had scored only 95 runs in five innings at this World Cup.",
    "India's first challenge is to balance the team, especially in deciding whether to include Mohammed Shami or R. Ashwin in the bowling lineup.",
    "India were likely to play an unchanged XI.",
    "India were without Hardik Pandya due to an ankle injury.",
    "England were likely to bring in Gus Atkinson for Mark Wood.",
    "England had just one win from five matches and were near the bottom of the points table.",
    "India's second challenge is to perform well if they have to bat first, as they have won all their matches in this World Cup by chasing.",
    "Shubman Gill, who missed the first two matches due to dengue, is expected to play a significant role."
]

In [41]:
final_scores_1, ranking_order_1, ranked_sentences_1 = test(testing_sentence_1)
metrics_1 = evaluate_ranking(ranked_sentences_1, testing_sentence_1)
print("Evaluation Metrics:")
for key, value in metrics_1.items():
    print(f"{key}: {value:.4f}")

c:\Users\draka\Downloads\Sports_Article_Ranking\virtual_environment\lib\site-packages\transformers\models\t5\tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Rank 1: Sentence: 'India and England will face each other in a blockbuster match on October 29, 2023, at the Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium in Lucknow.' - Score: 0.7365
Rank 2: Sentence: 'Historically, England has a better record against India in World Cup matches, winning four out of eight encounters, with India winning three and one match ending in a tie.' - Score: 0.6972
Rank 3: Sentence: 'India's second challenge is to perform well if they have to bat first, as they have won all their matches in this World Cup by chasing.' - Score: 0.6118
Rank 4: Sentence: 'England had just one win from five matches and were near the bottom of the points table.' - Score: 0.6079
Rank 5: Sentence: 'Captain Rohit Sharma emphasized India's focus on maintaining their winning streak.' - Score: 0.5958
Rank 6: Sentence: 'England has lost four out of their five matches so far and is almost out of the race for the semi-finals, whereas India has won all five of their matches.' - 

In [ ]:
# Chicago White Sox vs Texas Rangers MLB
testing_sentence_2 = [
    "Chicago, which has the worst record in baseball at 31-101 and is 4-30 since the All-Star break, is looking to end a four-game winless streak.",
    "Stats Insider's predictive analytics model gives the Rangers a 52% chance of winning against the White Sox at Guaranteed Rate Field.",
    "The White Sox are listed as 1.5-run underdogs against the Rangers, with odds of $1.71 available at TAB.",
    "According to Stats Insider's analytics model, the White Sox (+1.5) are predicted to cover the line 61% of the time, while the 8.5-run over/under is expected to stay under 55% of the time.",
    "White Sox starter Garrett Crochet threw four pitches \u0002014 two balls and two strikes \u00002014 before the game was halted.",
    "The Rangers entered the first game of the three-game series with a two-game losing streak.",
    "Stats Insider provides full betting coverage of the White Sox vs Texas game on Wednesday, including data-driven predictions and free betting tips.",
    "The Chicago White Sox will take on the Texas Rangers in MLB action at Guaranteed Rate Field on Wednesday, commencing at 10:10am AEST.",
    "The White Sox will send Garrett Crochet (6-9, 3.64 ERA) to the mound, up against Andrew Heaney (4-13, 4.04 ERA) for the Rangers.",
    "Based on trusted computer power and data, Stats Insider has simulated the result of Wednesday's White Sox vs Texas MLB match-up 10,000 times.",
    "The game between the White Sox and Rangers at Guaranteed Rate Field is scheduled to commence at 10:10am AEST on Wednesday.",
    "Andrew Heaney was the scheduled starter for Texas.",
    "The over/under for total runs scored is set at 8.5 with TAB, which has odds of $1.91 for the over and $1.91 for the under.",
    "For the favourite Rangers (-1.5) to cover the line, TAB offers odds of $2.14.",
    "The current betting odds in Australia for Wednesday's MLB game at Guaranteed Rate Field are: White Sox $2.15, Rangers $1.71.",
    "Teams: Chicago White Sox vs Texas Rangers",
    "Date: Wednesday 28 August 2024",
    "Time: 10:10am AEST",
    "Venue: Guaranteed Rate Field"
]

In [43]:
final_scores_2, ranking_order_2, ranked_sentences_2 = test(testing_sentence_2)
metrics_2 = evaluate_ranking(ranked_sentences_2, testing_sentence_2)
print("Evaluation Metrics:")
for key, value in metrics_2.items():
    print(f"{key}: {value:.4f}")

c:\Users\draka\Downloads\Sports_Article_Ranking\virtual_environment\lib\site-packages\transformers\models\t5\tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Rank 1: Sentence: 'Stats Insider provides full betting coverage of the White Sox vs Texas game on Wednesday, including data-driven predictions and free betting tips.' - Score: 0.6935
Rank 2: Sentence: 'Chicago, which has the worst record in baseball at 31-101 and is 4-30 since the All-Star break, is looking to end a four-game winless streak.' - Score: 0.6714
Rank 3: Sentence: 'Stats Insider's predictive analytics model gives the Rangers a 52% chance of winning against the White Sox at Guaranteed Rate Field.' - Score: 0.6206
Rank 4: Sentence: 'The Chicago White Sox will take on the Texas Rangers in MLB action at Guaranteed Rate Field on Wednesday, commencing at 10:10am AEST.' - Score: 0.5754
Rank 5: Sentence: 'The game between the White Sox and Rangers at Guaranteed Rate Field is scheduled to commence at 10:10am AEST on Wednesday.' - Score: 0.5401
Rank 6: Sentence: 'The current betting odds in Australia for Wednesday's MLB game at Guaranteed Rate Field are: White Sox $2.15, Rangers $1.7

In [ ]:
# New York Mets vs Boston Red Sox MLB
testing_sentence_3 = [
    "Recent performances show a stark contrast between the two teams. The Mets have been playing well, boasting a 7-3 record in their last ten games, with a solid .249 batting average and a 3.10 ERA. In contrast, the Red Sox are struggling, going 3-7 in their last ten, with a disappointing .199 batting average and a 3.24 ERA.",
    "The New York Mets have been able to leverage a potent offense and effective pitching to maintain their position in the playoff hunt.",
    "Historically, the Mets have had the advantage against the Red Sox in recent matchups, but with both teams fighting for playoff positioning, anything can happen.",
    "Pete Alonso has been a crucial player for the Mets this season. He has a batting average of .254 with 30 home runs and 80 RBIs.",
    "On the mound for the Red Sox will be Brayan Bello, who has a 12-6 record with a 4.66 ERA and a 1.37 WHIP.",
    "Luis Severino has recorded five or more strikeouts in each of his last six appearances with his team as a favorite against AL East opponents.",
    "J.D. Martinez has hit a home run in each of the Mets' last two night games against AL East opponents that held a winning record.",
    "Jeff McNeil has recorded at least one hit in each of his last 10 appearances against the Red Sox.",
    "The game is set to begin at 7:10 p.m. EDT on Monday night.",
    "This matchup marks the first time these teams will face each other this season.",
    "The Mets' lineup performs well against right-handed pitchers, enhancing Alonso's chances of capitalizing on any mistakes made by Bello.",
    "The Mets will start Luis Severino, who has a 9-6 record with a 3.96 ERA and a 1.26 WHIP.",
    "Severino has been effective in limiting runs and has shown the ability to strike out batters, making him a formidable opponent.",
    "Boston's bullpen has also struggled recently, providing further opportunities for Alonso to produce runs and hit home runs.",
    "Brandon Nimmo has recorded a Double in three of his last four appearances with the Mets as favorites against AL East opponents.",
    "The New York Mets (73-64) will host the Boston Red Sox (70-67) at Citi Field in Queens, New York, for the first game of a three-game series.",
    "Weather conditions at Citi Field are expected to be cloudy, with a temperature of 82°F, making for a comfortable evening for baseball.",
    "The 'Inning 1 OVER 0.5 runs' market has hit in each of the Mets' last four night games against American League opponents.",
    "The Boston Red Sox and the New York Mets meet Monday in MLB action from Citi Field.",
    "This will be the first installment in a three-game series.",
    "The Mets are currently third in the NL East, while the Red Sox sit third in the AL East.",
    "Alonso has shown remarkable power and consistency at the plate, making him a key player to watch in this matchup.",
    "Francisco Lindor has scored at least one run in eight of the Mets' last nine games as favorites against AL opponents.",
    "Masataka Yoshida has recorded at least one total base in each of his last 10 appearances with the Red Sox as underdogs.",
    "Tyler O'Neill has hit at least one home run in each of his last two appearances with the Red Sox against NL opponents.",
    "Jarren Duran has recorded a Double in each of the Red Sox's last four games as road underdogs against NL opponents.",
    "The Red Sox are going with Brayan Bello as the starting pitcher. In 25 starts this year, Bello is 12-6 with a 4.66 ERA.",
    "The 'Inning 1 UNDER 0.5 runs' market has hit in each of the Red Sox's last three games at Citi Field.",
    "Jarren Duran has scored at least one run in each of the Red Sox's last six games as underdogs against NL opponents.",
    "Jarren Duran has recorded at least one RBI in each of the Red Sox's last five games as road underdogs against NL opponents.",
    "The Mets' bullpen has been mostly reliable this season, with depth that allows them to manage leads effectively.",
    "The Red Sox bullpen has shown inconsistency, which could be a decisive factor in this matchup.",
    "Each of the Mets' last four games have gone UNDER the total runs line.",
    "Recent betting trends indicate that each of the Red Sox's last four night games against NL East opponents have gone OVER the total runs line."
]

In [80]:
final_scores_3, ranking_order_3, ranked_sentences_3 = test(testing_sentence_3)
metrics_3 = evaluate_ranking(ranked_sentences_3, testing_sentence_3)
print("Evaluation Metrics:")
for key, value in metrics_3.items():
    print(f"{key}: {value:.4f}")

Rank 1: Sentence: 'J.D. Martinez has hit a home run in each of the Mets' last two night games against AL East opponents that held a winning record.' - Score: 0.6635
Rank 2: Sentence: 'Recent performances show a stark contrast between the two teams. The Mets have been playing well, boasting a 7-3 record in their last ten games, with a solid .249 batting average and a 3.10 ERA. In contrast, the Red Sox are struggling, going 3-7 in their last ten, with a disappointing .199 batting average and a 3.24 ERA.' - Score: 0.6368
Rank 3: Sentence: 'Recent betting trends indicate that each of the Red Sox's last four night games against NL East opponents have gone OVER the total runs line.' - Score: 0.5749
Rank 4: Sentence: 'The 'Inning 1 OVER 0.5 runs' market has hit in each of the Mets' last four night games against American League opponents.' - Score: 0.5211
Rank 5: Sentence: 'The New York Mets (73-64) will host the Boston Red Sox (70-67) at Citi Field in Queens, New York, for the first game of a

In [ ]:
# Chicago Cubs vs Cincinnati Reds MLB
testing_sentence_4 = [
    "Stats Insider's predictive analytics model currently gives the Cubs a 57% chance of winning against the Reds at Wrigley Field.",
    "Jameson Taillon (11-8, 3.41 ERA) will get the ball for the Cubs, while the Reds are going with Nick Martinez (10-6, 3.22 ERA).",
    "Rhett Lowder is set to pitch for the Cincinnati Reds against the Chicago Cubs and Nico Hoerner.",
    "According to Stats Insider's model, the Reds (+1.5) are predicted to cover the line 62% of the time, while the 6.5-run over/under is expected to go over 59% of the time.",
    "The Chicago Cubs will take on the Cincinnati Reds in Game of the League Series at Wrigley Field on Saturday, commencing at 4:20am AEST.",
    "The latest betting odds in Australia for Saturday's MLB game at Wrigley Field are Cubs $1.75, Reds $2.05.",
    "The Cubs are listed as 1.5-run favourites against the Reds, with odds of $2.75 available at TAB.",
    "For the underdog Reds (+1.5) to cover the line, TAB offers odds of $1.45.",
    "The game between the Cubs and Reds at Wrigley Field is scheduled to start at 4:20am AEST on Saturday.",
    "The Cubs have a .243 batting average, ranking 14th in the majors.",
    "The Reds' pitching staff has an 8.7 K/9 this season, ranking 12th in the majors.",
    "The over/under for total runs scored is set at 6.5 with TAB, which has odds of $1.91 for the over and $1.91 for the under."
]

In [76]:
final_scores_4, ranking_order_4, ranked_sentences_4 = test(testing_sentence_4)
metrics_4 = evaluate_ranking(ranked_sentences_4, testing_sentence_4)
print("Evaluation Metrics:")
for key, value in metrics_4.items():
    print(f"{key}: {value:.4f}")

Rank 1: Sentence: 'The Chicago Cubs will take on the Cincinnati Reds in Game of the League Series at Wrigley Field on Saturday, commencing at 4:20am AEST.' - Score: 0.6147
Rank 2: Sentence: 'Stats Insider's predictive analytics model currently gives the Cubs a 57% chance of winning against the Reds at Wrigley Field.' - Score: 0.6048
Rank 3: Sentence: 'Jameson Taillon (11-8, 3.41 ERA) will get the ball for the Cubs, while the Reds are going with Nick Martinez (10-6, 3.22 ERA).' - Score: 0.5407
Rank 4: Sentence: 'The latest betting odds in Australia for Saturday's MLB game at Wrigley Field are Cubs $1.75, Reds $2.05.' - Score: 0.5255
Rank 5: Sentence: 'The game between the Cubs and Reds at Wrigley Field is scheduled to start at 4:20am AEST on Saturday.' - Score: 0.5169
Rank 6: Sentence: 'According to Stats Insider's model, the Reds (+1.5) are predicted to cover the line 62% of the time, while the 6.5-run over/under is expected to go over 59% of the time.' - Score: 0.5155
Rank 7: Sentence

In [ ]:
# Detroit Pistons vs Atlanta Hawks NBA
testing_sentence_5 = [
    "Stats Insider's predictive analytics model gives the Pistons a 58% chance of winning against the Hawks at Little Caesars Arena.",
    "The Detroit Pistons will face the Atlanta Hawks in NBA action at Little Caesars Arena on Saturday, starting at 11:00am AEDT.",
    "Stats Insider's predicted final score for Pistons vs Hawks at Little Caesars Arena on Saturday is the Pistons winning 114-112.",
    "The Pistons are 2-point favourites against the Hawks, with odds of $1.91 available at Bet365.",
    "Cade Cunningham is expected to lead the Pistons with 29 points, 6 rebounds and 8 assists, while Trae Young is projected to have 24 points, 3 rebounds and 11 assists for the Hawks.",
    "With the likes of Cade Cunningham, Jaden Ivey and Tobias Harris, the Pistons are up against a Hawks line-up that includes Trae Young, Jalen Johnson and Zaccharie Risacher.",
    "Detroit Pistons will host the Atlanta Hawks on November 9, 2024.",
    "Injuries for the Hawks include De'Andre Hunter (knee, day to day), Bogdan Bogdanovic (hamstring), Trae Young (rib, day to day), Kobe Bufkin (shoulder), and Vit Krejci (adductor).",
    "Injuries for the Pistons include Bobi Klintman (calf) and Ausar Thompson (illness).",
    "The over/under for total points scored is set at 227.5 with Bet365, which has odds of $1.91 for the over and $1.91 for the under.",
    "Detroit Pistons finished the 2023-24 season with a 14-68 record overall and 10-41 in Eastern Conference play.",
    "Atlanta Hawks finished the 2023-24 season with a 36-46 record overall and 22-30 in Eastern Conference play.",
    "Detroit Pistons averaged 109.9 points per game while shooting 46.3% from the field and 34.8% from behind the arc last season.",
    "Atlanta Hawks shot 46.5% from the field and 36.4% from 3-point range last season."
]

In [74]:
final_scores_5, ranking_order_5, ranked_sentences_5 = test(testing_sentence_5)
metrics_5 = evaluate_ranking(ranked_sentences_5, testing_sentence_5)
print("Evaluation Metrics:")
for key, value in metrics_5.items():
    print(f"{key}: {value:.4f}")

Rank 1: Sentence: 'Stats Insider's predictive analytics model gives the Pistons a 58% chance of winning against the Hawks at Little Caesars Arena.' - Score: 0.6433
Rank 2: Sentence: 'The Detroit Pistons will face the Atlanta Hawks in NBA action at Little Caesars Arena on Saturday, starting at 11:00am AEDT.' - Score: 0.6331
Rank 3: Sentence: 'Stats Insider's predicted final score for Pistons vs Hawks at Little Caesars Arena on Saturday is the Pistons winning 114-112.' - Score: 0.6148
Rank 4: Sentence: 'With the likes of Cade Cunningham, Jaden Ivey and Tobias Harris, the Pistons are up against a Hawks line-up that includes Trae Young, Jalen Johnson and Zaccharie Risacher.' - Score: 0.5994
Rank 5: Sentence: 'Detroit Pistons finished the 2023-24 season with a 14-68 record overall and 10-41 in Eastern Conference play.' - Score: 0.5950
Rank 6: Sentence: 'Atlanta Hawks finished the 2023-24 season with a 36-46 record overall and 22-30 in Eastern Conference play.' - Score: 0.5859
Rank 7: Senten

In [ ]:
# Golden State Warriors vs New Orleans Pelicans NBA
testing_sentence_6 = [
    "Draymond Green recorded a game-high five blocks, four of which were on Zion Williamson, who was held to the lowest field goal percentage of his career (25 percent, 5-for-20).",
    "New Orleans (2-3) dropped a third consecutive game after their 2-0 start on Wednesday with a 104-89 loss at Golden State (4-1).",
    "Jordan Hawkins put up a game-high 23 points for New Orleans.",
    "Jordan Hawkins had 23 points for the Pelicans.",
    "Green's final block came on a Brandon Ingram 3-pointer.",
    "Draymond Green contributed 14 points, six assists, four rebounds, and five blocks.",
    "Jose Alvarado had 16 points and Brandon Ingram 14 points with six assists.",
    "Brandin Podziemski nearly recorded a triple-double with 13 points, eight rebounds, and seven assists.",
    "Trayce Jackson-Davis had 15 points and nine rebounds.",
    "Trayce Jackson-Davis added 15 points and nine rebounds.",
    "James Wenn had 17 points.",
    "Brandin Podziemski finished with 13 points, nine rebounds, and eight assists.",
    "Zion Williamson collected 12 rebounds to complement 12 points.",
    "Buddy Hield scored 21 points.",
    "Buddy Hield had 21 points to lead six scorers in double figures.",
    "Kyle Anderson scored 10 points.",
    "Jonathan Kuminga had 16 points.",
    "Jordan Hawkins put up his second 20+ point outing on the season, scoring 23 points."
]

In [51]:
final_scores_6, ranking_order_6, ranked_sentences_6 = test(testing_sentence_6)
metrics_6 = evaluate_ranking(ranked_sentences_6, testing_sentence_6)
print("Evaluation Metrics:")
for key, value in metrics_6.items():
    print(f"{key}: {value:.4f}")

c:\Users\draka\Downloads\Sports_Article_Ranking\virtual_environment\lib\site-packages\transformers\models\t5\tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Rank 1: Sentence: 'New Orleans (2-3) dropped a third consecutive game after their 2-0 start on Wednesday with a 104-89 loss at Golden State (4-1).' - Score: 0.6701
Rank 2: Sentence: 'Draymond Green recorded a game-high five blocks, four of which were on Zion Williamson, who was held to the lowest field goal percentage of his career (25 percent, 5-for-20).' - Score: 0.6006
Rank 3: Sentence: 'Buddy Hield had 21 points to lead six scorers in double figures.' - Score: 0.5040
Rank 4: Sentence: 'Jose Alvarado had 16 points and Brandon Ingram 14 points with six assists.' - Score: 0.4625
Rank 5: Sentence: 'Jordan Hawkins put up his second 20+ point outing on the season, scoring 23 points.' - Score: 0.4548
Rank 6: Sentence: 'Brandin Podziemski nearly recorded a triple-double with 13 points, eight rebounds, and seven assists.' - Score: 0.4429
Rank 7: Sentence: 'Kyle Anderson scored 10 points.' - Score: 0.4411
Rank 8: Sentence: 'Buddy Hield scored 21 points.' - Score: 0.4345
Rank 9: Sentence: 'Gr

In [ ]:
# India vs England Odi
testing_sentence_7 = [
    "India's victory will galvanize the team as they head into the business end of the tournament.",
    "Rohit Sharma, adjudged the player of the match, emphasized the importance of his partnership with KL Rahul and taking the game deep.",
    "Rohit Sharma was named Player of the Match for his 87 runs.",
    "Rohit Sharma expressed pride in the team's character and experience, noting the challenge of batting first and the effectiveness of their bowling attack.",
    "Rohit Sharma: 'This was a game that showed a lot of character in our squad. When times were tough, our experienced players stood up at the right time and fought for us.'",
    "England's captain Jos Buttler will have much to contemplate as their chances of progression look even more scant.",
    "Jos Buttler expressed disappointment in England's performance, particularly their batting, despite a good start by the bowlers.",
    "Jos Buttler: 'Very disappointing. At the halfway stage chasing 230 we fancied ourselves. But it's the same old story.'",
    "Player of the match Rohit Sharma: 'Looking at where we were after the first 10 overs of our batting, it was important to put on a partnership like myself and KL Rahul did. It was a challenging pitch to start with but it got easier the longer you spent in the middle. We are very happy with the performance.'",
    "Rohit Sharma felt India were 30-40 runs short but was pleased with the overall performance.",
    "India earned 2 points from the match.",
    "Chris Woakes expressed disappointment and mentioned the lack of confidence in the team.",
    "Jasprit Bumrah mentioned the team's enjoyment of the tournament and their practice in evening conditions.",
    "Eoin Morgan commented on England's underperformance, suggesting something is unsettled within the team.",
    "Jos Buttler acknowledged the team's poor performance and the need to play for Champions Trophy qualification."
]

In [55]:
final_scores_7, ranking_order_7, ranked_sentences_7 = test(testing_sentence_7)
metrics_7 = evaluate_ranking(ranked_sentences_7, testing_sentence_7)
print("Evaluation Metrics:")
for key, value in metrics_7.items():
    print(f"{key}: {value:.4f}")

Rank 1: Sentence: 'Player of the match Rohit Sharma: 'Looking at where we were after the first 10 overs of our batting, it was important to put on a partnership like myself and KL Rahul did. It was a challenging pitch to start with but it got easier the longer you spent in the middle. We are very happy with the performance.'' - Score: 0.6956
Rank 2: Sentence: 'Rohit Sharma, adjudged the player of the match, emphasized the importance of his partnership with KL Rahul and taking the game deep.' - Score: 0.5501
Rank 3: Sentence: 'Jos Buttler acknowledged the team's poor performance and the need to play for Champions Trophy qualification.' - Score: 0.5429
Rank 4: Sentence: 'Jos Buttler expressed disappointment in England's performance, particularly their batting, despite a good start by the bowlers.' - Score: 0.5148
Rank 5: Sentence: 'Rohit Sharma was named Player of the Match for his 87 runs.' - Score: 0.4897
Rank 6: Sentence: 'England's captain Jos Buttler will have much to contemplate as

In [ ]:
# India vs Afghanistan T20i
testing_sentence_8 = [
    "India and Afghanistan tied the match, leading to a double Super Over, which India won by 10 runs.",
    "India won the match after two thrilling super overs.",
    "India and Afghanistan were forced to take the field for a Super Over after both sides scored 212 runs each in the third T20I at Bengaluru's M Chinnaswamy Stadium.",
    "In the second Super Over, India scored 11 runs while Afghanistan managed only 1 run, resulting in India's victory.",
    "Ravi Bishnoi took 2 wickets in the second super over to secure the win for India.",
    "Rohit Sharma hit two sixes in the first Super Over and another six in the second Super Over.",
    "Rohit Sharma retired himself out to get a better runner at the non-striker's end in the first Super Over.",
    "The Super Over also ended in a draw with both sides scoring 16 runs each.",
    "Fareed Ahmad got Rinku Singh out on review and then Rohit Sharma ran himself out trying to steal a bye in the second Super Over.",
    "Rohit Sharma came back to bat in the second Super Over and hit a six and a four.",
    "Mukesh Kumar bowled straight yorkers successfully in the first Super Over.",
    "India scored 36 runs in the final over of the match.",
    "Rohit Sharma scored 121 runs off 69 balls with 11 fours and eight sixes.",
    "Rohit Sharma hit two sixes in the first Super Over.",
    "Ravi Bishnoi took two crucial wickets in the second Super Over.",
    "Virat Kohli was incensed and clapped in Nabi's face after an overthrow off the body in the first Super Over.",
    "Virat Kohli made a crucial fielding effort to save a potential six in the 17th over.",
    "Rinku Singh played an unbeaten knock of 69 runs from 39 balls with two fours and six sixes.",
    "Fareed Ahmad took 3 wickets for 20 runs.",
    "Rohit Sharma scored an unbeaten 121 off 58 balls.",
    "Rohit Sharma scored an unbeaten 121 off 69 balls.",
    "Gulbadin Naib scored an unbeaten 55 off 23 balls.",
    "Rinku Singh scored an unbeaten 69 off 39 balls.",
    "Washington Sundar took 3 wickets for 18 runs.",
    "India equalled Afghanistan's score of 16 in the first super over.",
    "Fareed Ahmad Malik took 3 wickets for 20 runs.",
    "Rinku Singh scored 69* off 36 balls.",
    "Gulbadin Naib scored 55 off 23 balls.",
    "Afghanistan's Rahmanullah Gurbaz and Ibrahim Zadran both scored half-centuries.",
    "Mohammad Nabi contributed 34 runs off 16 balls.",
    "Rahmanullah Gurbaz scored 50 runs.",
    "India won the match in the 2nd Super Over.",
    "Gulbadin Naib scored an unbeaten 55."
]

In [57]:
final_scores_8, ranking_order_8, ranked_sentences_8 = test(testing_sentence_8)
metrics_8 = evaluate_ranking(ranked_sentences_8, testing_sentence_8)
print("Evaluation Metrics:")
for key, value in metrics_8.items():
    print(f"{key}: {value:.4f}")

c:\Users\draka\Downloads\Sports_Article_Ranking\virtual_environment\lib\site-packages\transformers\models\t5\tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Rank 1: Sentence: 'India and Afghanistan were forced to take the field for a Super Over after both sides scored 212 runs each in the third T20I at Bengaluru's M Chinnaswamy Stadium.' - Score: 0.7865
Rank 2: Sentence: 'Rohit Sharma hit two sixes in the first Super Over and another six in the second Super Over.' - Score: 0.7623
Rank 3: Sentence: 'Ravi Bishnoi took 2 wickets in the second super over to secure the win for India.' - Score: 0.7379
Rank 4: Sentence: 'Rohit Sharma retired himself out to get a better runner at the non-striker's end in the first Super Over.' - Score: 0.7176
Rank 5: Sentence: 'Virat Kohli was incensed and clapped in Nabi's face after an overthrow off the body in the first Super Over.' - Score: 0.7089
Rank 6: Sentence: 'In the second Super Over, India scored 11 runs while Afghanistan managed only 1 run, resulting in India's victory.' - Score: 0.6590
Rank 7: Sentence: 'Virat Kohli made a crucial fielding effort to save a potential six in the 17th over.' - Score: 0.

In [ ]:
# Argentina vs Chile Soccer
testing_sentence_9 = [
    "Argentina defeated Chile 3-0 in World Cup qualifying on Thursday.",
    "Alexis Mac Allister, Julian Alvarez and Paulo Dybala were on target as Argentina tightened their grip on South America's 2026 World Cup qualifying tournament with a 3-0 win over Chile.",
    "Argentina claimed a 3-0 home win over Chile thanks to second-half goals from Alexis Mac Allister, Julian Alvarez, and Paulo Dybala.",
    "Three second half goals saw Argentina ease past Chile in World Cup Qualifying.",
    "Goals from Alexis Mac Allister, Julian Alvarez, and Paulo Dybala sealed the victory.",
    "Former Manchester City striker Alvarez doubled the lead in the 84th minute with a thumping shot from outside the area before substitute Dybala put the seal on the victory with a late strike in stoppage time.",
    "Liverpool midfielder Mac Allister swept in a low finish three minutes after half-time to break the deadlock at the Estadio Monumental in Buenos Aires against a defensive Chile side.",
    "Liverpool midfielder Mac Allister scored three minutes after the break off a cross from Alvarez.",
    "Scorers: Mac Allister 48′, Alvarez 84′, Dybala 90+1",
    "Alexis Mac Allister popped up from midfield to fire Argentina into a deserved lead.",
    "Julián Álvarez got his name on the scoresheet with a screamer from the edge of the box.",
    "Julian Alvarez scored a long-range stunner in the 84th minute.",
    "Substitute Paulo Dybala found the net in stoppage time to seal the victory.",
    "Julian Alvarez scored a long-range effort in the 84th minute.",
    "Paulo Dybala added a third goal in injury time.",
    "Paulo Dybala scored the third goal in stoppage-time.",
    "Julian Alvarez scored in the 84th minute.",
    "Paulo Dybala scored in the 90'+1' minute.",
    "Alexis Mac Allister scored in the 48th minute.",
    "Alexis Mac Allister scored the opening goal three minutes into the second half."
]

In [97]:
final_scores_9, ranking_order_9, ranked_sentences_9 = test(testing_sentence_9)
metrics_9 = evaluate_ranking(ranked_sentences_9, testing_sentence_9)
print("Evaluation Metrics:")
for key, value in metrics_9.items():
    print(f"{key}: {value:.4f}")

c:\Users\draka\Downloads\Sports_Article_Ranking\virtual_environment\lib\site-packages\transformers\models\t5\tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Rank 1: Sentence: 'Alexis Mac Allister, Julian Alvarez and Paulo Dybala were on target as Argentina tightened their grip on South America's 2026 World Cup qualifying tournament with a 3-0 win over Chile.' - Score: 0.8435
Rank 2: Sentence: 'Argentina claimed a 3-0 home win over Chile thanks to second-half goals from Alexis Mac Allister, Julian Alvarez, and Paulo Dybala.' - Score: 0.8099
Rank 3: Sentence: 'Liverpool midfielder Mac Allister swept in a low finish three minutes after half-time to break the deadlock at the Estadio Monumental in Buenos Aires against a defensive Chile side.' - Score: 0.6812
Rank 4: Sentence: 'Former Manchester City striker Alvarez doubled the lead in the 84th minute with a thumping shot from outside the area before substitute Dybala put the seal on the victory with a late strike in stoppage time.' - Score: 0.6099
Rank 5: Sentence: 'Paulo Dybala added a third goal in injury time.' - Score: 0.5755
Rank 6: Sentence: 'Three second half goals saw Argentina ease pas

In [ ]:
# Armenia vs Latvia Soccer
testing_sentence_10 = [
    "Armenia have lost their last five Nations League matches, and conceded 17 goals in those fixtures.",
    "In the opening game of their 2024-25 UEFA Nations League C campaign, Armenia are set to welcome Latvia at Vazgen Sargsyan anvan Hanrapetakan Marzadasht on Saturday.",
    "Boss Oleksandr Petrakov's side come into Saturday's clash having lost seven of their last nine games, as well as having failed to win nine of their last 10 fixtures.",
    "The visitors are currently ranked 137th in the world, and in their most recent Nations League campaign in 2022-23, they were promoted from the D League into the C League.",
    "The home side beat Kazakhstan 2-1 last time out on June 7, while the visitors managed to overcome Faroe Islands 1-0 on June 11.",
    "Latvia will have to improve on their away form if they are to win on Saturday given they are winless in their last eight games on the road, losing six of those clashes.",
    "Opponents Latvia narrowly beat Faroe Islands 1-0 thanks to a goal in the 50th minute from Andrejs Ciganiks.",
    "Armenia have at least scored in 10 of their last 14 games in all competitions.",
    "Latvia are not regarded as good travelers, and the prediction is that Armenia will emerge triumphant in the match.",
    "Armenia continues life without Henrikh Mkhitaryan, who retired from international football, and nowadays captain Varazdat Haroyan is the team’s most influential player.",
    "A key player to watch in the Latvian team is Daejeon attacker Vladislavs Gutkovskis, who has netted 11 goals in 45 appearances for the national team.",
    "Latvia struggled in the EURO 2024 qualifiers, collecting only three points in total.",
    "Hovhannisyan, Bichakhchyan, and M’gladbach forward Ranos made the cut for the Armenian team.",
    "Boss Paolo Nicolato's side have failed to win their two most recent matches in the competition, but prior to these results they did win their last five Nations League games.",
    "Armenia will look to kick off their Nations League campaign in style when they face Latvia at Vazgen Sargsyan Republican Stadium."
]

In [72]:
final_scores_10, ranking_order_10, ranked_sentences_10 = test(testing_sentence_10)
metrics_10 = evaluate_ranking(ranked_sentences_10, testing_sentence_10)
print("Evaluation Metrics:")
for key, value in metrics_10.items():
    print(f"{key}: {value:.4f}")

Rank 1: Sentence: 'Boss Paolo Nicolato's side have failed to win their two most recent matches in the competition, but prior to these results they did win their last five Nations League games.' - Score: 0.8794
Rank 2: Sentence: 'Armenia continues life without Henrikh Mkhitaryan, who retired from international football, and nowadays captain Varazdat Haroyan is the team’s most influential player.' - Score: 0.6998
Rank 3: Sentence: 'Latvia will have to improve on their away form if they are to win on Saturday given they are winless in their last eight games on the road, losing six of those clashes.' - Score: 0.6183
Rank 4: Sentence: 'A key player to watch in the Latvian team is Daejeon attacker Vladislavs Gutkovskis, who has netted 11 goals in 45 appearances for the national team.' - Score: 0.6003
Rank 5: Sentence: 'Boss Oleksandr Petrakov's side come into Saturday's clash having lost seven of their last nine games, as well as having failed to win nine of their last 10 fixtures.' - Score:

# Top NDCS@10

In [81]:
# metrics_1, metrics_2, ..., metrics_10

all_metrics = [metrics_1, metrics_2, metrics_3, metrics_4, metrics_5,
               metrics_6, metrics_7, metrics_8, metrics_9, metrics_10]

# Sort the metrics by NDCG@10 in descending order
sorted_metrics = sorted(all_metrics, key=lambda m: m['NDCG@10'], reverse=True)

# Select the top 3 based on NDCG@10
top3_metrics = sorted_metrics[:3]

In [82]:
# Display the top 3 metrics
print("Top 3 runs based on NDCG@10:")
for i, m in enumerate(top3_metrics, start=1):
    print(f"Run {i}:")
    for key, value in m.items():
        print(f"  {key}: {value:.4f}")
    print()

# Compute the average metrics across these top 3 runs
final_metrics = {}
for key in top3_metrics[0].keys():
    final_metrics[key] = sum(m[key] for m in top3_metrics) / len(top3_metrics)

print("Final averaged metrics (from top 3 runs):")
for key, value in final_metrics.items():
    print(f"{key}: {value:.4f}")

Top 3 runs based on NDCG@10:
Run 1:
  NDCG@5: 0.9577
  NDCG@10: 0.9620
  Recall@5: 0.6000
  Recall@10: 0.7000

Run 2:
  NDCG@5: 0.6590
  NDCG@10: 0.8246
  Recall@5: 0.2000
  Recall@10: 0.5000

Run 3:
  NDCG@5: 0.7492
  NDCG@10: 0.7592
  Recall@5: 0.4000
  Recall@10: 0.6000

Final averaged metrics (from top 3 runs):
NDCG@5: 0.7887
NDCG@10: 0.8486
Recall@5: 0.4000
Recall@10: 0.6000


# Weights using NN with GPT4o ranked sentence with some score value as well

In [83]:
import os
import json
import openai
import torch
import torch.nn as nn
import torch.optim as optim

In [98]:
import json
import numpy as np
import re

#########################################
# 1. Parsing and Extraction Helpers
#########################################

def make_array(data):
    """
    Extracts sentences and scores from data in one of three formats:
    
    Format 1: {"sentences": [ { "sentence": "...", "score": ... }, ... ]}
    Format 2: {"ranked_sentences": [ { sentence: score }, ... ]}
    Format 3: Flat dictionary: { sentence: score, ... }
    
    Parameters:
        data (dict): The JSON data loaded as a dictionary.
    
    Returns:
        tuple: Two lists (sentences, scores)
    """
    # Format 1: Using the "sentences" key.
    if isinstance(data, dict) and "sentences" in data and isinstance(data["sentences"], list):
        sentences = []
        scores = []
        for item in data["sentences"]:
            if isinstance(item, dict) and "sentence" in item and "score" in item:
                sentences.append(item["sentence"])
                scores.append(item["score"])
    # Format 2: Using the "ranked_sentences" key.
    elif isinstance(data, dict) and "ranked_sentences" in data and isinstance(data["ranked_sentences"], list):
        sentences = []
        scores = []
        for item in data["ranked_sentences"]:
            if isinstance(item, dict):
                for sentence, score in item.items():
                    sentences.append(sentence)
                    scores.append(score)
    # Format 3: Assume a flat dictionary: { sentence: score, ... }
    elif isinstance(data, dict):
        sentences = list(data.keys())
        scores = list(data.values())
    else:
        sentences, scores = [], []
    
    return sentences, scores

def parse_ranked_json(file_path):
    """
    Parses a ranked JSON file and returns a list of (sentence, score) pairs.
    Supports multiple formats:
      - Format 1: {"sentences": [ { "sentence": "...", "score": ... }, ... ]}
      - Format 2: {"ranked_sentences": [ { sentence: score }, ... ]}
      - Format 3: Flat dictionary: { sentence: score, ... }
    """
    try:
        with open(file_path, "r") as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return []
    
    sentences, scores = make_array(data)
    return list(zip(sentences, scores))

def is_number(x):
    """Return True if x can be converted to a float."""
    try:
        float(x)
        return True
    except:
        return False

def extract_numeric(s):
    """
    Uses regex to find the first numeric value in a string.
    Returns the number as a float if found, else None.
    """
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    if match:
        try:
            return float(match.group())
        except:
            return None
    return None

def extract_sentences_and_scores(file_path):
    """
    Extracts sentences and scores from a ranked JSON file.
    
    Returns:
      - sentences: A NumPy array of sentences (strings).
      - scores: A NumPy array of scores as floats.
    
    This function handles the following formats:
      1. {"sentences": [ { "sentence": "...", "score": ... }, ... ]}
      2. {"ranked_sentences": [ { sentence: score }, ... ]}
      3. { sentence: score, ... }
    
    If a score cannot be directly converted, it will attempt to extract a numeric substring.
    """
    pairs = parse_ranked_json(file_path)
    sentences = []
    scores = []
    for a, b in pairs:
        score_val = None
        try:
            # Heuristic: if one element is numeric and the other is not,
            # assign the numeric element as the score.
            if is_number(a) and not is_number(b):
                score_val = float(a)
                sent = str(b).strip()
            elif is_number(b) and not is_number(a):
                score_val = float(b)
                sent = str(a).strip()
            else:
                # Otherwise, assume a is the sentence and b is the score.
                sent = str(a).strip()
                score_val = float(b)
        except Exception as e:
            # If conversion fails, try to extract a number from the candidate score field.
            score_val = extract_numeric(str(b))
            if score_val is None:
                # Try the other element.
                score_val = extract_numeric(str(a))
                if score_val is None:
                    print(f"Skipping pair ({a}, {b}) in {file_path}: could not extract numeric score.")
                    continue
                else:
                    sent = str(b).strip()
            else:
                sent = str(a).strip()
        sentences.append(sent)
        scores.append(score_val)
    return np.array(sentences), np.array(scores)

def filter_sentences_and_scores(sentences, scores):
    """
    Applies the sanitize_sentences function to filter out any sentences that don't pass.
    Returns filtered lists of sentences and scores (with corresponding indices aligned).
    """
    filtered_sentences = []
    filtered_scores = []
    for s, sc in zip(sentences, scores):
        # Assume sanitize_sentences is defined elsewhere.
        sanitized = sanitize_sentences([s])
        if sanitized and sanitized[0].strip() == s.strip():
            filtered_sentences.append(s)
            filtered_scores.append(sc)
    return filtered_sentences, filtered_scores

In [99]:
#########################################
# 2. Feature Matrix Computation
#########################################

def create_score_matrix(sentences):
    """
    Computes a score matrix for a list of sentences.
    
    Each row corresponds to a sentence and columns represent scores:
      [Semantic, Sentiment, TF-IDF, Buzzword, NER People, Sarcasm]
    
    Assumes the helper functions and global variables below are defined elsewhere:
      - sanitize_sentences
      - process_sentence(s, verb_dict, embedding_model, index, verbs, nlp)
      - detect_emotion_intensity(sentences)
      - tfidf_sums(sentences)
      - calculate_buzzword_score(s, sports_keywords)
      - calculate_ner_people_score(s, nlp, famous_people)
      - get_sarcasm_score([s])
      
    Parameters:
      - sentences: List of sentence strings.
      
    Returns:
      - A NumPy array representing the score matrix.
    """
    scores_matrix = []
    
    # Sanitize sentences.
    sanitized = sanitize_sentences(sentences)
    
    semantic_scores = [process_sentence(s, verb_dict, embedding_model, index, verbs, nlp) for s in sanitized]
    sentiment_scores_list = detect_emotion_intensity(sanitized)
    tfidf_scores = tfidf_sums(sanitized)
    buzzword_scores = [calculate_buzzword_score(s, sports_keywords) for s in sanitized]
    ner_people_scores = [calculate_ner_people_score(s, nlp, famous_people) for s in sanitized]
    sarcasm_scores = [get_sarcasm_score([s])[0] for s in sanitized]
    
    for i in range(len(sanitized)):
        scores_matrix.append([
            semantic_scores[i],
            sentiment_scores_list[i],
            tfidf_scores[i],
            buzzword_scores[i],
            ner_people_scores[i],
            sarcasm_scores[i]
        ])
    
    return np.array(scores_matrix)

In [109]:
#########################################
# 3. Training Data Collection
#########################################

def collect_training_data(root_folder):
    """
    Collects training data from the ranked outputs in the "GPT4o_scored" folders
    from all available matches or datasets inside the root folder.
    
    For each ranked JSON file, it extracts sentences and scores using the helpers,
    filters them via sanitize_sentences, computes the feature matrix via create_score_matrix,
    and then pairs each sentence's feature vector with its normalized overall score.
    
    Returns:
      - training_data: List of tuples (feature_vector, target_score)
    """
    training_data = []

    for folder in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder)
        if not os.path.isdir(folder_path):
            continue
        for match in os.listdir(folder_path):
            match_path = os.path.join(folder_path, match)
            if not os.path.isdir(match_path):
                continue
            ranked_folder = os.path.join(match_path, "GPT4o_scored")
            if not os.path.exists(ranked_folder):
                continue
            for cat in os.listdir(ranked_folder):
                cat_folder = os.path.join(ranked_folder, cat)
                if not os.path.isdir(cat_folder):
                    continue
                for timeline in ["before", "after"]:
                    timeline_folder = os.path.join(cat_folder, timeline)
                    if not os.path.isdir(timeline_folder):
                        continue
                    ranked_file = os.path.join(timeline_folder, "ranked.json")
                    if not os.path.exists(ranked_file):
                        continue
                    # Extract sentences and scores.
                    sentences, scores = extract_sentences_and_scores(ranked_file)
                    if len(sentences) == 0:
                        continue
                    # Filter using sanitize_sentences.
                    sentences, scores = filter_sentences_and_scores(sentences, scores)
                    if len(sentences) == 0:
                        continue
                    # Normalize scores (assumed to be between 0 and 10).
                    target_scores = [float(sc)/10.0 for sc in scores]
                    # Compute the feature matrix from the sentences.
                    features_matrix = create_score_matrix(sentences)
                    n = min(len(features_matrix), len(target_scores))
                    for i in range(n):
                        feature_vector = features_matrix[i].tolist()
                        target = target_scores[i]
                        training_data.append((feature_vector, target))
    return training_data

In [110]:
#########################################
# 4. Neural Network (Single Neuron Perceptron)
#########################################

class ScoreNet(nn.Module):
    def __init__(self):
        super(ScoreNet, self).__init__()
        # Use logits so that softmax over them gives non-negative weights summing to 1.
        # Initialize logits to zeros so that softmax(logits) = [1/6, ..., 1/6].
        self.logits = nn.Parameter(torch.zeros(6))
    
    def forward(self, x):
        weights = torch.softmax(self.logits, dim=0)
        return (x * weights).sum()

def train_nn_model(training_data, model=None, epochs=10, learning_rate=0.01):
    """
    Trains the ScoreNet model on the provided training data.
    
    Parameters:
      - training_data: List of tuples (feature_vector, target_score).
      - model: An existing ScoreNet instance to update; if None, a new one is created.
      - epochs: Number of epochs (one full pass over all training samples).
      - learning_rate: Learning rate for training.
      
    Returns:
      - model: The updated ScoreNet model.
      
    After each epoch, prints the average loss and current non-negative weights.
    """
    if model is None:
        model = ScoreNet()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    
    for epoch in range(epochs):
        epoch_loss = 0.0
        for x, y in training_data:
            if not torch.is_tensor(x):
                x = torch.tensor(x, dtype=torch.float32)
            if not torch.is_tensor(y):
                y = torch.tensor([y], dtype=torch.float32)
                
            optimizer.zero_grad()
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        current_weights = torch.softmax(model.logits, dim=0).detach().numpy()
        print(f"Epoch {epoch+1}: Average Loss = {epoch_loss/len(training_data):.4f}")
        print(f"Weights after epoch {epoch+1}: {current_weights}")
    
    final_weights = torch.softmax(model.logits, dim=0).detach().numpy()
    print("Final Weights:", final_weights)
    return model

In [111]:
#########################################
# 5. Automated Training Function
#########################################

def update_weights_from_all_data(root_folder, model=None, epochs=50, learning_rate=0.01):
    """
    Collects training data from the ranked outputs in the "ranked_gpt" folders
    (from MLB and specified NBA matches), then trains the ScoreNet model on that data.
    
    Parameters:
      - root_folder: Path to the root folder containing sports folders.
      - model: An existing ScoreNet model to update; if None, a new model is created.
      - epochs: Number of training epochs.
      - learning_rate: Learning rate for training.
      
    Returns:
      - The updated ScoreNet model.
    """
    training_data = collect_training_data(root_folder)
    if not training_data:
        print("No training data available from root folder:", root_folder)
        return model
    updated_model = train_nn_model(training_data, model=model, epochs=epochs, learning_rate=learning_rate)
    return updated_model

In [117]:
#########################################
# 6. Main Execution
#########################################

if __name__ == "__main__":
    # Set the path to your root folder containing sports folders.
    root_folder = "GPT4o/GPT4o"  # Adjust this path as needed.
    
    # Automatically update weights using the collected training data.
    model = update_weights_from_all_data(root_folder, None, epochs=50, learning_rate=0.001)
    
    if model is not None:
        final_weights = torch.softmax(model.logits, dim=0).detach().numpy()
        print("Updated Weights:", final_weights)


Epoch 1: Average Loss = 0.0394
Weights after epoch 1: [0.34535575 0.3960107  0.07932178 0.06653231 0.05432962 0.05844981]
Epoch 2: Average Loss = 0.0289
Weights after epoch 2: [0.3588582  0.39338708 0.07962708 0.0558328  0.05187748 0.06041739]
Epoch 3: Average Loss = 0.0288
Weights after epoch 3: [0.363181   0.39037296 0.08185536 0.05065026 0.05049076 0.06344976]
Epoch 4: Average Loss = 0.0288
Weights after epoch 4: [0.36436957 0.38940087 0.08375291 0.04749697 0.04895467 0.06602512]
Epoch 5: Average Loss = 0.0288
Weights after epoch 5: [0.3644788  0.38913795 0.08527062 0.04544052 0.04742003 0.06825208]
Epoch 6: Average Loss = 0.0288
Weights after epoch 6: [0.36430463 0.38905242 0.08645364 0.04403537 0.04595481 0.07019916]
Epoch 7: Average Loss = 0.0288
Weights after epoch 7: [0.36413744 0.38895428 0.08736812 0.04304301 0.04458611 0.07191104]
Epoch 8: Average Loss = 0.0288
Weights after epoch 8: [0.3640625  0.38879225 0.08807757 0.04232476 0.04332171 0.07342121]
Epoch 9: Average Loss = 

# Test of NN obtained weights

In [113]:
def test_nn(sentences):
    """
    Creates a score matrix from sentences, computes final scores using final weights,
    ranks the sentences in descending order based on their scores, and prints the ranking.
    
    Returns:
      final_scores: NumPy array of computed scores.
      ranked_indices: Indices of sentences sorted by score (descending).
      ranked_sentences: List of sentences sorted by rank.
    """
    # Get score matrix: each row corresponds to one sentence with 6 feature scores.
    score_matrix = create_score_matrix(sentences)
    
    # Final weights (normalized so that sum(weights)=1)
    final_weights = np.array([0.36969835, 0.38172263, 0.09224957, 0.03982576, 0.02984306, 0.08666058])
    
    # Compute final score for each sentence (weighted sum of feature scores)
    final_scores = score_matrix.dot(final_weights)
    
    # Rank sentence indices in descending order (higher score is better)
    ranked_indices = final_scores.argsort()[::-1]
    
    # Order sentences based on ranking
    ranked_sentences = [sentences[idx] for idx in ranked_indices]
    
    # Print ranking information: rank, sentence, and score
    for rank, idx in enumerate(ranked_indices, start=1):
        print(f"Rank {rank}: Sentence: '{sentences[idx]}' - Score: {final_scores[idx]:.4f}")
    
    return final_scores, ranked_indices, ranked_sentences

# Example usage:
# sentences = ["Sentence one text.", "Sentence two text.", "Sentence three text.", ...]
# final_scores, ranking_order, ranked_sentences = test_nn(sentences)


In [114]:
# Using the neural network variant of the test function (test_nn) for all test cases.
# For each test case, we create new variables with the "_nn" suffix, compute evaluation metrics,
# and print them out.

# Test 1: India vs England ODI
final_scores_1_nn, ranking_order_1_nn, ranked_sentences_1_nn = test_nn(testing_sentence_1)
metrics_1_nn = evaluate_ranking(ranked_sentences_1_nn, testing_sentence_1)
print("Evaluation Metrics for Test 1 NN:")
for key, value in metrics_1_nn.items():
    print(f"{key}: {value:.4f}")

# Test 2: Chicago White Sox vs Texas Rangers MLB
final_scores_2_nn, ranking_order_2_nn, ranked_sentences_2_nn = test_nn(testing_sentence_2)
metrics_2_nn = evaluate_ranking(ranked_sentences_2_nn, testing_sentence_2)
print("\nEvaluation Metrics for Test 2 NN:")
for key, value in metrics_2_nn.items():
    print(f"{key}: {value:.4f}")

# Test 3: New York Mets vs Boston Red Sox MLB
final_scores_3_nn, ranking_order_3_nn, ranked_sentences_3_nn = test_nn(testing_sentence_3)
metrics_3_nn = evaluate_ranking(ranked_sentences_3_nn, testing_sentence_3)
print("\nEvaluation Metrics for Test 3 NN:")
for key, value in metrics_3_nn.items():
    print(f"{key}: {value:.4f}")

# Test 4: Chicago Cubs vs Cincinnati Reds MLB
final_scores_4_nn, ranking_order_4_nn, ranked_sentences_4_nn = test_nn(testing_sentence_4)
metrics_4_nn = evaluate_ranking(ranked_sentences_4_nn, testing_sentence_4)
print("\nEvaluation Metrics for Test 4 NN:")
for key, value in metrics_4_nn.items():
    print(f"{key}: {value:.4f}")

# Test 5: Detroit Pistons vs Atlanta Hawks NBA
final_scores_5_nn, ranking_order_5_nn, ranked_sentences_5_nn = test_nn(testing_sentence_5)
metrics_5_nn = evaluate_ranking(ranked_sentences_5_nn, testing_sentence_5)
print("\nEvaluation Metrics for Test 5 NN:")
for key, value in metrics_5_nn.items():
    print(f"{key}: {value:.4f}")

# Test 6: Golden State Warriors vs New Orleans Pelicans NBA
final_scores_6_nn, ranking_order_6_nn, ranked_sentences_6_nn = test_nn(testing_sentence_6)
metrics_6_nn = evaluate_ranking(ranked_sentences_6_nn, testing_sentence_6)
print("\nEvaluation Metrics for Test 6 NN:")
for key, value in metrics_6_nn.items():
    print(f"{key}: {value:.4f}")

# Test 7: India vs England ODI
final_scores_7_nn, ranking_order_7_nn, ranked_sentences_7_nn = test_nn(testing_sentence_7)
metrics_7_nn = evaluate_ranking(ranked_sentences_7_nn, testing_sentence_7)
print("\nEvaluation Metrics for Test 7 NN:")
for key, value in metrics_7_nn.items():
    print(f"{key}: {value:.4f}")

# Test 8: India vs Afghanistan T20I
final_scores_8_nn, ranking_order_8_nn, ranked_sentences_8_nn = test_nn(testing_sentence_8)
metrics_8_nn = evaluate_ranking(ranked_sentences_8_nn, testing_sentence_8)
print("\nEvaluation Metrics for Test 8 NN:")
for key, value in metrics_8_nn.items():
    print(f"{key}: {value:.4f}")

# Test 9: Argentina vs Chile Soccer
final_scores_9_nn, ranking_order_9_nn, ranked_sentences_9_nn = test_nn(testing_sentence_9)
metrics_9_nn = evaluate_ranking(ranked_sentences_9_nn, testing_sentence_9)
print("\nEvaluation Metrics for Test 9 NN:")
for key, value in metrics_9_nn.items():
    print(f"{key}: {value:.4f}")

# Test 10: Armenia vs Latvia Soccer
final_scores_10_nn, ranking_order_10_nn, ranked_sentences_10_nn = test_nn(testing_sentence_10)
metrics_10_nn = evaluate_ranking(ranked_sentences_10_nn, testing_sentence_10)
print("\nEvaluation Metrics for Test 10 NN:")
for key, value in metrics_10_nn.items():
    print(f"{key}: {value:.4f}")


Rank 1: Sentence: 'India and England will face each other in a blockbuster match on October 29, 2023, at the Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium in Lucknow.' - Score: 0.8438
Rank 2: Sentence: 'Historically, England has a better record against India in World Cup matches, winning four out of eight encounters, with India winning three and one match ending in a tie.' - Score: 0.8404
Rank 3: Sentence: 'India's second challenge is to perform well if they have to bat first, as they have won all their matches in this World Cup by chasing.' - Score: 0.8005
Rank 4: Sentence: 'England has lost four out of their five matches so far and is almost out of the race for the semi-finals, whereas India has won all five of their matches.' - Score: 0.7850
Rank 5: Sentence: 'Captain Rohit Sharma emphasized India's focus on maintaining their winning streak.' - Score: 0.7788
Rank 6: Sentence: 'Jos Buttler had scored only 95 runs in five innings at this World Cup.' - Score: 0.7543
Rank

# Top 3 metrics @NDCG for NN obtained weights

In [115]:
# Assuming metrics_1_nn, metrics_2_nn, ..., metrics_10_nn are already defined

all_metrics_nn = [metrics_1_nn, metrics_2_nn, metrics_3_nn, metrics_4_nn, metrics_5_nn,
                  metrics_6_nn, metrics_7_nn, metrics_8_nn, metrics_9_nn, metrics_10_nn]

# Sort the NN metrics by NDCG@10 in descending order
sorted_metrics_nn = sorted(all_metrics_nn, key=lambda m: m['NDCG@10'], reverse=True)

# Select the top 3 based on NDCG@10
top3_metrics_nn = sorted_metrics_nn[:3]

# Display the top 3 NN metrics
print("Top 3 NN runs based on NDCG@10:")
for i, m in enumerate(top3_metrics_nn, start=1):
    print(f"Run {i}:")
    for key, value in m.items():
        print(f"  {key}: {value:.4f}")
    print()

# Compute the average metrics across these top 3 NN runs
final_metrics_nn = {}
for key in top3_metrics_nn[0].keys():
    final_metrics_nn[key] = sum(m[key] for m in top3_metrics_nn) / len(top3_metrics_nn)

print("Final averaged NN metrics (from top 3 runs):")
for key, value in final_metrics_nn.items():
    print(f"{key}: {value:.4f}")


Top 3 NN runs based on NDCG@10:
Run 1:
  NDCG@5: 0.6767
  NDCG@10: 0.8250
  Recall@5: 0.4000
  Recall@10: 0.4000

Run 2:
  NDCG@5: 0.7786
  NDCG@10: 0.7780
  Recall@5: 0.8000
  Recall@10: 0.7000

Run 3:
  NDCG@5: 0.7474
  NDCG@10: 0.7374
  Recall@5: 0.4000
  Recall@10: 0.3000

Final averaged NN metrics (from top 3 runs):
NDCG@5: 0.7342
NDCG@10: 0.7801
Recall@5: 0.5333
Recall@10: 0.4667


In [116]:
# Compare top 3 runs for both regular and NN metrics side by side

print("Comparison of Top 3 Runs (SGD vs NN) based on NDCG@10:\n")
for i in range(3):
    print(f"Run {i+1}:")
    regular_run = top3_metrics[i]
    nn_run = top3_metrics_nn[i]
    # Assuming both dictionaries share the same keys
    for key in regular_run.keys():
        reg_value = regular_run[key]
        nn_value = nn_run[key]
        print(f"  {key}: SGD = {reg_value:.4f}   |   NN = {nn_value:.4f}")
    print()

# Compute final averaged metrics for both Regular and NN top 3 runs
final_metrics_regular = {}
final_metrics_nn = {}
for key in top3_metrics[0].keys():
    final_metrics_regular[key] = sum(m[key] for m in top3_metrics) / len(top3_metrics)
    final_metrics_nn[key] = sum(m[key] for m in top3_metrics_nn) / len(top3_metrics_nn)

print("Final Averaged Metrics Comparison (SGD vs NN):")
for key in final_metrics_regular.keys():
    print(f"{key}: SGD = {final_metrics_regular[key]:.4f}   |   NN = {final_metrics_nn[key]:.4f}")


Comparison of Top 3 Runs (SGD vs NN) based on NDCG@10:

Run 1:
  NDCG@5: SGD = 0.9577   |   NN = 0.6767
  NDCG@10: SGD = 0.9620   |   NN = 0.8250
  Recall@5: SGD = 0.6000   |   NN = 0.4000
  Recall@10: SGD = 0.7000   |   NN = 0.4000

Run 2:
  NDCG@5: SGD = 0.6590   |   NN = 0.7786
  NDCG@10: SGD = 0.8246   |   NN = 0.7780
  Recall@5: SGD = 0.2000   |   NN = 0.8000
  Recall@10: SGD = 0.5000   |   NN = 0.7000

Run 3:
  NDCG@5: SGD = 0.7492   |   NN = 0.7474
  NDCG@10: SGD = 0.7592   |   NN = 0.7374
  Recall@5: SGD = 0.4000   |   NN = 0.4000
  Recall@10: SGD = 0.6000   |   NN = 0.3000

Final Averaged Metrics Comparison (SGD vs NN):
NDCG@5: SGD = 0.7887   |   NN = 0.7342
NDCG@10: SGD = 0.8486   |   NN = 0.7801
Recall@5: SGD = 0.4000   |   NN = 0.5333
Recall@10: SGD = 0.6000   |   NN = 0.4667
